In [1]:
# import libraries
import numpy as np

In [98]:
""" create datasets """

def create_dataset(examples_num):
    
    # create X dataset
    X = np.random.rand(dataset_X_size[0], dataset_X_size[1])
    X1 = X.copy()

    # transform X into a dataset with only 0 & 1
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            if X[i][j] < 0.5:
                X1[i][j] = 0
            else:
                X1[i][j] = 1

    # create Y XOR dataset
    Y = np.logical_xor(X1[0], X1[1])

#     X = np.random.randint(0, 2, size = (2, examples_num))

#     Y = X.sum(axis=0, keepdims = True)  
#     Y[Y != 1] = 0
    
    return X, Y




In [3]:
dataset_X_size = (2,100)
print(dataset_X_size[0], dataset_X_size[1])
# X, Y = create_dataset(dataset_X_size)
# print(X,Y)

2 100


In [297]:
""" define mathematical activation function and their derivatives """

# relu function
def relu(Z):
    return np.maximum(0,Z)

# sigmoid function
def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

# relu backward function / derivative
def relu_back(dA):
    dZ = np.array(dA, copy=True)       
    dZ[dZ < 0] = 0
    return dZ

# sigmoid backward function / derivative
def sigmoid_back(dA, A):
    dZ = dA * (A * (1 - A))
    assert dZ.shape == A.shape    
    return dZ

In [5]:
# double check relu function
A = relu(Z)
print (A)

# double check sigmoid function
A = sigmoid(Z)
print(A)

# double check relu backward function
dA = np.sum(- np.divide(Y, A) + np.divide(1 - Y, 1 - A))
dZ = relu_back(dA)
print(dZ)

# double check sigmoid backward function
dZ = sigmoid_back(A)
print (dZ)

NameError: name 'Z' is not defined

In [244]:
""" 1 - initialize parameters """

def initialize_parameters(layer_dims):
    
    parameters = {}
    L = len(layer_dims)
    
    for i in range(1, L):
        parameters['W'+str(i)] = np.random.randn(layer_dims[i], layer_dims[i-1]) * 0.01
        parameters['b'+str(i)] = np.random.randn(layer_dims[i], 1)
        
        assert parameters['W'+str(i)].shape == (layer_dims[i], layer_dims[i-1])
        assert parameters['b'+str(i)].shape == (layer_dims[i], 1)
    
    return parameters

In [7]:
# double check initialize_parameters function
parameters = initialize_parameters([len(X), 3, 3, 1])
print(parameters)

NameError: name 'X' is not defined

In [8]:
""" 2 - forward propagation """



# 2.1 - forward linear function

def forward_linear(A, W, b):
    
    linear_cache = []
    
    Z = np.dot(W, A) + b
    linear_cache = [A, W, b]
    
    assert Z.shape == (W.shape[0], A.shape[1])
        
    return Z, linear_cache



# 2.2 - forward activation function

def foward_activation(A_prev, W, b, activation):

    activation_cache = []
    cache = []
    Z, linear_cache = forward_linear(A_prev, W, b)
    
    if activation == 'relu':
        A = relu(Z)
    elif activation == 'sigmoid':
        A = sigmoid(Z)
    else:
        print("activation function non-recognised")
        
    assert A.shape == Z.shape
    
    activation_cache = [Z]
    cache = (linear_cache, activation_cache)
        
    return A, cache



# 2.3 - forward propagation (L-layer)

def forward_L_layer(X, parameters):
    
    caches = []
    L = len(parameters) // 2
    A_prev = X
    
    for i in range(1,L):
        W = parameters['W' + str(i)]
        b = parameters['b' + str(i)]
        A_prev, cache = foward_activation(A_prev, W, b, activation='relu')
        caches.append(cache)
        
    
    W = parameters['W' + str(L)]
    b = parameters['b' + str(L)]
    A, cache = foward_activation(A_prev, W, b, activation='sigmoid')
    caches.append(cache)
    
    return (A, caches)

In [9]:
# double check forward linear function

Z, linear_cache = forward_linear(X, parameters['W1'], parameters['b1'])
# print(Z)
print(parameters['W1'])
print(parameters['b1'])
print(linear_cache)

NameError: name 'X' is not defined

In [10]:
# double check forward activation function
A_prev = X
A, cache = foward_activation(A_prev, parameters['W1'], parameters['b1'], activation='sigmoid')
# print(A, cache)

NameError: name 'X' is not defined

In [11]:
# double check forward propagation (L-layer)

A, caches = forward_L_layer(X, parameters)
print(A)
print(A.shape)

NameError: name 'X' is not defined

In [183]:
""" 3 - cost function """

def cost_function(Y, A):

    cost = 0
    m = A.shape[1]
    cost = - np.sum(Y * np.log(A) + (1 - Y) * np.log(1 - A)) / m
    
#     J = np.squeeze(J)   
#     assert J.shape == ()
    
    return cost

In [13]:
# double check cost function

J = cost_function(Y, A)
print(type(J))
print(J)

NameError: name 'Y' is not defined

In [299]:
""" 4 - backward propagation """



# 4.1 - backward linear function

def back_linear(linear_cache, dZ):
    
    A_prev, W, b = linear_cache
    m = A_prev.shape[1]
    
    dA_prev = np.dot(W.T, dZ)
    dW = np.dot(dZ, A_prev.T) / m
    db = np.sum(dZ, axis=1, keepdims=True) / m
    
    return dA_prev, dW, db



# 4.2 - backward activation function

def back_activation(dA, A, caches, activation):
    
    linear_cache, activation_cache = caches
    
    if activation == 'relu_back':
        dZ = relu_back(dA)
    elif activation == 'sigmoid_back':
        dZ = sigmoid_back(dA, A)
    else:
        print("activation function non-recognised")
        
    dA_prev, dW, db = back_linear(linear_cache, dZ)
    
    return dA_prev, dW, db



# 4.3 - backward propagation (L-layer)

def backward_L_layer(A, Y, caches):
    
    grads = {}
    L = len(caches)
    Y = Y.reshape(A.shape)
    
    dA = - np.divide(Y, A) + np.divide(1 - Y, 1 - A)

    dA_prev, dW, db = back_activation(dA, A, caches[L-1], activation='sigmoid_back')
#     grads['dA_prev'+str(L-1)] = dA_prev
    grads['dW'+str(L)] = dW
    grads['db'+str(L)] = db

    
    for i in reversed(range(L-1)):
        dA_prev, dW_temps, db_temps = back_activation(dA_prev, A, caches[i], activation='relu_back')
#         dA_prev, dW, db = back_activation(grads['dA_prev'+str(i+1)], A, caches[i], activation='relu_back')
#         grads['dA_prev'+str(i)] = dA_prev
        grads['dW'+str(i+1)] = dW_temps
        grads['db'+str(i+1)] = db_temps
    
    return grads

In [15]:
# double check the whole backward propagation function
grads = backward_L_layer(A, Y, caches)
print(grads)

NameError: name 'A' is not defined

In [321]:
""" 5 - update parameters function """

def update_parameters(parameters, grads, learning_rate):
    
    L = len(parameters) // 2
    
    for i in range(L):

        dW = grads['dW'+str(i+1)]
        db = grads['db'+str(i+1)]
        
        parameters['W'+str(i+1)] -= learning_rate * dW
        parameters['b'+str(i+1)] -= learning_rate * db
        
    return parameters

In [17]:
print(parameters)
learning_rate = 0.1
parameters = update_parameters(parameters, grads, learning_rate)
print(parameters)

NameError: name 'parameters' is not defined

In [327]:
""" running the DNN model by Channing"""

def L_layer_DNN(dataset_X_size, iteration_num, layer_dims, learning_rate):

    
    # 1 - initialize parameter
    parameters = initialize_parameters(layer_dims)
#     print('parameters_initial: ', parameters)
    
    for i in range(iteration_num):

        # 2 - forward propagation
        A, caches = forward_L_layer(X, parameters)

        # 3 - cost function 
        cost = cost_function(Y, A)

        # 4 - backward propagation
        grads = backward_L_layer(A, Y, caches)

        # 5 - update parameters function
        parameters = update_parameters(parameters, grads, learning_rate)
#         if i == iteration_num-1:
#             print('parameters_updated: ', parameters)

        if i % 100 == 0:
            print ('cost', i, ': ', cost)
        
#     return X, Y, parameters, A, caches, cost
    return parameters

In [338]:
# create datasets
examples_num = 400
X, Y = create_dataset(dataset_X_size)

# calling the DNN model
iteration_num = 3000
# layer_dims = [len(X), 4, 3, 1]
layer_dims = [len(X), 100, 1]
learning_rate = 0.01

parameters = L_layer_DNN(examples_num, iteration_num, layer_dims, learning_rate)

cost 0 :  0.9200686619626447
cost 100 :  0.693027643761415
cost 200 :  0.6930229955634869
cost 300 :  0.6930186963632874
cost 400 :  0.6930151772000709
cost 500 :  0.6930125016885907
cost 600 :  0.6930107386830942
cost 700 :  0.6930099625951496
cost 800 :  0.6930102536959802
cost 900 :  0.6930116984179862
cost 1000 :  0.6930143896561302
cost 1100 :  0.6930184270662627
cost 1200 :  0.6930239173571957
cost 1300 :  0.6930309750425758
cost 1400 :  0.6930397278067774
cost 1500 :  0.6930503192001592
cost 1600 :  0.6930628944745322
cost 1700 :  0.6930776005932496
cost 1800 :  0.6930945813748217
cost 1900 :  0.6931139721210806
cost 2000 :  0.693135912463427
cost 2100 :  0.6931605690248612
cost 2200 :  0.6931881336489256
cost 2300 :  0.69321882369518
cost 2400 :  0.6932528684877576
cost 2500 :  0.6932905008785257
cost 2600 :  0.6933319584875699
cost 2700 :  0.6933776554561004
cost 2800 :  0.6934285975398756
cost 2900 :  0.6934859201981155
